In [151]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize
import thor.attribute as attribute

notebook_name = "cpol_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T16:00:00"
end = "2005-11-13T18:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory / "options")

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

grid_options = grid.create_options(name="geographic")
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
# visualize_options["middle_cloud"] = visualize.option.runtime_options(
#     "middle_cloud", save=True, style="presentation", figure_types=["mask"]
# )
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-09-26 11:58:35,233 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-09-26 11:58:35,275 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-26 11:58:35,276 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-09-26 11:58:35,279 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-09-26 11:58:35,299 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml


In [158]:
if output_directory.exists():
    shutil.rmtree(output_directory / "attributes")
    shutil.rmtree(output_directory / "masks")
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-09-26 11:58:35,468 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-26 11:58:35,558 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-09-26 11:58:35,559 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-09-26 11:58:35,560 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-09-26 11:58:35,618 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-09-26 11:58:36,283 - thor.track - INFO - Processing hierarchy level 0.
2024-09-26 11:58:36,285 - thor.track - INFO - Tracking cell.
2024-09-26 11:58:36,332 - thor.match.match - INFO - Matching cell objects.
2024-09-26 11:58:36,334 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-26 11:58:36,344 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-26 11:58:36,346 - thor.track - INFO - Tracking middle_cloud.
2024-09-26 11:58:36,358 - thor.visualize.runt

# Cartesian Coordinates

In [142]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory / "options")

grid_options = grid.create_options(name="cartesian", regrid=False)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data.option.save_data_options(data_options, options_directory)
option.save_track_options(track_options, options_directory)
# visualize.option.save_display_options(visualize_options, options_directory)


2024-09-26 11:41:01,301 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-26 11:41:01,303 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/grid.yml
2024-09-26 11:41:01,308 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/data.yml
2024-09-26 11:41:01,351 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/track.yml
2024-09-26 11:41:01,700 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/visualize.yml


In [143]:
if output_directory.exists():
    shutil.rmtree(output_directory / "attributes")
    shutil.rmtree(output_directory / "masks")
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-09-26 11:41:01,795 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-26 11:41:01,871 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-09-26 11:41:01,872 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-09-26 11:41:01,874 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-09-26 11:41:02,046 - thor.track - INFO - Processing hierarchy level 0.
2024-09-26 11:41:02,049 - thor.track - INFO - Tracking cell.
2024-09-26 11:41:02,081 - thor.match.match - INFO - Matching cell objects.
2024-09-26 11:41:02,084 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-26 11:41:02,096 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-26 11:41:02,098 - thor.track - INFO - Tracking middle_cloud.
2024-09-26 11:41:02,121 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-26 11:41:02,124 - thor.track - INFO - Tracking anvil.


# Analysis

In [159]:
import thor.analyze as analyze

In [168]:
options = analyze.utils.read_options(output_directory)
track_options = options["track"]
cell_bounds = track_options[0]["cell"]["detection"]["altitudes"]
cell_bounds


[500, 3000]

In [169]:
winds = attribute.utils.read_attribute_csv(output_directory / "attributes/cell/profile.csv")[["u", "v"]]

In [171]:
import pandas as pd


In [192]:
cell_wind = winds.loc[pd.IndexSlice[:, :, cell_bounds[0]:cell_bounds[1]], :]
cell_mean_wind = cell_wind.groupby(level=["time", "universal_id"]).mean()
cell_top = winds.xs(cell_bounds[1], level="altitude")
cell_bottom = winds.xs(cell_bounds[0], level="altitude")
cell_shear = cell_top - cell_bottom